# Lesson 3: Intermediate Prompt Engineering and Task-Specific Strategies

## Setup
First, let's set up our environment. Run this cell before proceeding with the examples.

In [ ]:
# Run this cell to setup the OpenAI client (please be patient, during initial setup)
exec('''import importlib.util\nimport subprocess\nimport sys\nimport os\nfrom getpass import getpass\n_client = None\ndef check_openai_installed():\n    if importlib.util.find_spec('openai') is None:\n        print('OpenAI package not found. Installing...')\n        subprocess.check_call([sys.executable, '-m', 'pip', 'install',\n            'openai'])\n        print('OpenAI package installed successfully.')\n    else:\n        print('OpenAI package is already installed.')\ndef import_openai():\n    check_openai_installed()\n    global OpenAI\n    from openai import OpenAI\ndef validate_api_key(api_key):\n    try:\n        client = OpenAI(api_key=api_key)\n        client.models.list()\n        return True\n    except Exception as e:\n        print(f'Error validating API key: {str(e)}')\n        return False\ndef get_api_key():\n    while True:\n        api_key = os.getenv('OPENAI_API_KEY')\n        if not api_key:\n            print('OPENAI_API_KEY not found in environment variables.')\n            api_key = getpass('Please enter your OpenAI API key: ')\n        if validate_api_key(api_key):\n            os.environ['OPENAI_API_KEY'] = api_key\n            return api_key\n        else:\n            print('Invalid API key. Please try again.')\n            os.environ.pop('OPENAI_API_KEY', None)\ndef setup_openai_client():\n    global _client\n    import_openai()\n    api_key = get_api_key()\n    _client = OpenAI(api_key=api_key)\n    print(f"API Key set: {'Yes' if _client.api_key else 'No'}")\n    print(\n        f"API Key (first 5 chars): {_client.api_key[:5] if _client.api_key else 'None'}"\n        )\n    return _client\ndef get_completion(prompt, model='gpt-3.5-turbo'):\n    global _client\n    if _client is None:\n        _client = setup_openai_client()\n    messages = [{'role': 'user', 'content': prompt}]\n    response = _client.chat.completions.create(model=model, messages=\n        messages, temperature=0)\n    return response.choices[0].message.content''')
client = setup_openai_client()

# ## 1. Multi-Step Text Analysis

# This function performs a multi-step analysis of a given text:
# 1. It summarizes the text in one sentence.
# 2. It identifies three key themes based on the summary.
# 3. It generates three thought-provoking questions for further analysis based on the themes.
# The function uses the get_completion() method to interact with the GPT model for each step.
# It returns a string containing the summary, themes, and questions.

In [ ]:
def analyze_text(text):
    # Step 1: Summarize the text
    summary_prompt = f"Summarize the following text in one sentence: {text}"
    summary = get_completion(summary_prompt)
    
    # Step 2: Identify key themes
    themes_prompt = f"Based on this summary: '{summary}', identify three key themes."
    themes = get_completion(themes_prompt)
    
    # Step 3: Generate questions for further analysis
    questions_prompt = f"Given these themes: {themes}, generate three thought-provoking questions for further analysis."
    questions = get_completion(questions_prompt)
    
    return f"Summary: {summary}\n\nThemes: {themes}\n\nQuestions for Analysis: {questions}"

sample_text = """
Climate change is one of the most pressing issues of our time. It affects every aspect of our lives, from the food we eat to the air we breathe. Scientists agree that human activities, particularly the burning of fossil fuels, are the primary cause of the rapid warming we're seeing. The consequences are far-reaching, including rising sea levels, more frequent and severe weather events, and disruptions to ecosystems worldwide. Addressing this challenge requires global cooperation and significant changes to our energy systems and lifestyles.
"""

print(analyze_text(sample_text))

## 2. Prompt Optimization Techniques
This cell compares different prompting strategies for explaining AI, demonstrating how specificity, analogies, and Socratic questioning affect AI-generated responses. The 'compare_prompt_variations' function enables direct comparison of these strategies, highlighting the crucial role of prompt formulation in shaping AI outputs.

In [ ]:
def compare_prompt_variations(base_prompt, variations):
    results = {}
    for name, prompt in variations.items():
        results[name] = get_completion(prompt)
    return results

base_prompt = "Explain the concept of artificial intelligence."
variations = {
    "Basic": base_prompt,
    "Specific": "Explain the concept of artificial intelligence, focusing on machine learning and neural networks.",
    "Analogical": "Explain the concept of artificial intelligence by comparing it to human intelligence.",
    "Socratic": "What is artificial intelligence? How does it work? What are its implications?",
}

results = compare_prompt_variations(base_prompt, variations)
for name, result in results.items():
    print(f"{name} Prompt Result:\n{result}\n")

## 3. Task-Specific Prompting Strategies
These cells introduce task-specific prompting strategies, which are tailored approaches for common AI tasks like text summarization, code debugging, and data interpretation. These strategies optimize the AI's performance for particular use cases, improving the quality and relevance of responses.

In [ ]:
# Text Summarization
def summarize_text(text, max_words=50):
    prompt = f"Summarize the following text in no more than {max_words} words:\n\n{text}"
    return get_completion(prompt)

text_to_summarize = "Your long text here..."
print("Summary:", summarize_text(text_to_summarize))

In [ ]:
# Code Debugging
def debug_code(code, error_message):
    prompt = f"""
    The following code is producing an error:
    
    ```
    {code}
    ```
    
    Error message: {error_message}
    
    Please identify the issue and suggest a fix.
    """
    return get_completion(prompt)

code_to_debug = """
def calculate_average(numbers):
    total = sum(numbers)
    average = total / len(numbers)
    return average

result = calculate_average([])
print(result)
"""
print("Debugging result:", debug_code(code_to_debug, "ZeroDivisionError: division by zero"))

In [ ]:
# Data Interpretation
def interpret_data(data):
    prompt = f"""
    Analyze the following data and provide key insights:
    
    {data}
    
    Please include:
    1. Main trends
    2. Any anomalies
    3. Possible implications
    """
    return get_completion(prompt)

data_to_interpret = """
Year, Sales (millions)
2018, 1.5
2019, 2.2
2020, 1.8
2021, 3.1
2022, 3.8
"""
print("Data interpretation:", interpret_data(data_to_interpret))

## 4. Handling Ambiguity and Uncertainty
Demonstrate strategies for prompting the model to express uncertainty or provide multiple perspectives.

In [ ]:
def generate_balanced_response(topic):
    prompt = f"""
    Provide a balanced perspective on the topic of {topic}. 
    Include:
    1. At least two different viewpoints
    2. Potential areas of uncertainty or debate
    3. Any caveats or limitations in our current understanding
    
    Clearly indicate when you're expressing uncertainty or presenting conflicting views.
    """
    return get_completion(prompt)

print(generate_balanced_response("the impact of artificial intelligence on job markets"))

## 5. Interactive and Iterative Prompting
Show how to design prompts that facilitate back-and-forth interactions with the model.

In [ ]:
def interactive_story_generation():
    context = "You are a creative writing assistant helping to craft an interactive story."
    story = "Once upon a time, in a dense forest..."
    
    for _ in range(3):  # Simulate 3 interactions
        prompt = f"""
        {context}
        Current story: {story}
        
        Provide two possible continuations for this story, each no more than 50 words.
        Label them as Option A and Option B.
        """
        options = get_completion(prompt)
        print(options)
        
        choice = input("Choose A or B to continue the story: ").upper()
        if choice not in ['A', 'B']:
            print("Invalid choice. Ending the story.")
            break
        
        continuation_prompt = f"""
        {context}
        Current story: {story}
        Chosen continuation: Option {choice} from the following options:
        {options}
        
        Please continue the story based on Option {choice}, adding about 50 words.
        """
        continuation = get_completion(continuation_prompt)
        story += " " + continuation
        print(f"\nUpdated story:\n{story}\n")

interactive_story_generation()

## Conclusion and Exercises
In this lesson, we've explored advanced prompt engineering techniques and task-specific strategies. These methods allow for more sophisticated interactions with AI models, enabling complex task decomposition, optimized prompts for better results, and handling of ambiguity and uncertainty.

## Exercises

### Exercise 1: Task Decomposition for Complex Problem Solving

Implement a function that uses task decomposition to solve a complex math word problem. The function should break down the problem into smaller steps, solve each step, and then combine the results for the final answer.

1. Define a function `solve_complex_math_problem(problem)` that takes a word problem as input.
2. Use the Chain of Thought technique to break down the problem into steps.
3. Solve each step using the GPT model.
4. Combine the results to get the final answer.
5. Test your function with a complex word problem.

In [ ]:
def solve_complex_math_problem(problem):
    # Your code here
    pass

# Test your function
test_problem = """
A store is having a 20% off sale. Alice buys a shirt for $25 and a pair of jeans for $45. 
Bob buys two shirts and a pair of jeans. How much more does Bob spend than Alice after the discount?
"""
solution = solve_complex_math_problem(test_problem)
print(solution)

### Exercise 2: Optimizing Prompts for Specific Tasks

Create a function that generates optimized prompts for different types of writing tasks. The function should take the task type and key information as input, and return a well-structured prompt.

1. Define a function `generate_optimized_prompt(task_type, key_info)` that takes the task type (e.g., "essay", "report", "story") and key information as input.
2. Implement different prompt structures for each task type.
3. Use the key information to customize the prompt.
4. Test your function with at least two different task types.

In [ ]:
def generate_optimized_prompt(task_type, key_info):
    # Your code here
    pass

# Test your function
essay_prompt = generate_optimized_prompt("essay", {"topic": "Climate Change", "word_count": 500})
print("Essay Prompt:")
print(essay_prompt)

story_prompt = generate_optimized_prompt("story", {"genre": "Science Fiction", "main_character": "Time Traveler"})
print("\nStory Prompt:")
print(story_prompt)

### Exercise 3: Handling Ambiguity and Uncertainty

Extend the `generate_balanced_response` function to include a confidence score for different aspects of the response. The function should provide a more nuanced view of the topic by indicating the level of certainty for various claims or viewpoints.

1. Modify the `generate_balanced_response` function to include confidence scores.
2. Update the prompt to ask the model to provide confidence levels (e.g., high, medium, low) for different aspects of the response.
3. Parse the response to extract the main points and their associated confidence levels.
4. Present the balanced view with confidence indicators.
5. Test your updated function with a controversial or complex topic.

In [ ]:
def generate_balanced_response_with_confidence(topic):
    # Your code here
    pass

# Test your function
response = generate_balanced_response_with_confidence("the effectiveness of renewable energy sources")
print(response)